In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
lead_lag_sess = SparkSession.builder.appName("Lead_Lag_Example").getOrCreate()
df_spark = lead_lag_sess.read.csv("train.csv", header=True, inferSchema=True)
lead_lag_window = Window.partitionBy('Store').orderBy('Date')

25/11/24 18:56:07 WARN Utils: Your hostname, Shithils-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/11/24 18:56:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 18:56:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/24 18:56:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/24 18:56:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df_spark=df_spark.withColumn('Prev_Day_Sales', lag(col('Sales'),1).over(lead_lag_window)).withColumn('Sales_Diff', col('Sales') - col('Prev_Day_Sales')).withColumn('Percentage_Diff', (col('Sales') - col('Prev_Day_Sales'))/col('Prev_Day_Sales')*100)
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Prev_Day_Sales|Sales_Diff|   Percentage_Diff|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|          NULL|      NULL|              NULL|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|             0|      4958|              NULL|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|          4958|       329|  6.63574021782977|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|          5287|       194|3.6693777189332324|
|   28|        6|2013-01-05| 2070|      287|   1|    0|           0|            0|          5481|     -3

In [4]:
df_spark=df_spark.withColumn('Growth_Flag', when(col('Sales')>col('Prev_Day_Sales'),lit(1)).otherwise(lit(0)))
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+-----------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Prev_Day_Sales|Sales_Diff|   Percentage_Diff|Growth_Flag|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+-----------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|          NULL|      NULL|              NULL|          0|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|            1|             0|      4958|              NULL|          1|
|   28|        4|2013-01-03| 5287|      641|   1|    0|           0|            1|          4958|       329|  6.63574021782977|          1|
|   28|        5|2013-01-04| 5481|      623|   1|    0|           0|            1|          5287|       194|3.6693777189332324|          1|
|   28|        6|201

In [11]:
df_spark=df_spark.withColumn('Next_Day_Sales', lead(col('Sales'),1).over(lead_lag_window)).withColumn('Next_2Day_Sales', lead(col('Sales'),2).over(lead_lag_window)).withColumn('Next_3Day_Sales', lead(col('Sales'),3).over(lead_lag_window)).withColumn('Avg_Next3Days_Sales', (col('Next_Day_Sales') + col('Next_2Day_Sales') + col('Next_3Day_Sales'))/3)
df_spark=df_spark.withColumn('Future_3Day_Growth_Flag', when(col('Avg_Next3Days_Sales')>col('Sales'),lit(1)).otherwise(lit(0)))
df_spark.show(5)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+-----------+--------------+---------------+---------------+-------------------+-----------------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Prev_Day_Sales|Sales_Diff|   Percentage_Diff|Growth_Flag|Next_Day_Sales|Next_2Day_Sales|Next_3Day_Sales|Avg_Next3Days_Sales|Future_3Day_Growth_Flag|
+-----+---------+----------+-----+---------+----+-----+------------+-------------+--------------+----------+------------------+-----------+--------------+---------------+---------------+-------------------+-----------------------+
|   28|        2|2013-01-01|    0|        0|   0|    0|           a|            1|          NULL|      NULL|              NULL|          0|          4958|           5287|           5481|             5242.0|                      1|
|   28|        3|2013-01-02| 4958|      609|   1|    0|           0|        